In [1]:
# Dash and JupyterDash framework for interactive dashboard in notebook environment
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State

# Standard Python and system libraries
import base64
import os

# Data processing and visualization libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

# Custom CRUD module for MongoDB database access
from animal_crud import AnimalShelter

In [2]:
from pymongo import MongoClient

# MongoDB connection settings
username = "aacuser"
password = "123456"
host = "nv-desktop-services.apporto.com"
port = 31295
db_name = "AAC"
collection_name = "animals"

# Create client and connect
client = MongoClient(f"mongodb://{username}:{password}@{host}:{port}/?authSource=admin")
db = client[db_name]
collection = db[collection_name]

# Read and format data
data = list(collection.find())
df = pd.DataFrame(data)

# Drop _id column to prevent crash in Dash
if "_id" in df.columns:
    df.drop(columns=['_id'], inplace=True)



In [3]:
# Dashboard Layout / View
from jupyter_dash import JupyterDash
from dash import html, dcc, dash_table
import base64

# Load custom logo and encode for embedding
image_filename = 'Grazioso Salvare Logo.png'  # Ensure this file is in the working directory
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# Initialize Dash app
app = JupyterDash(__name__)

# Define app layout
app.layout = html.Div([

    # Animal type filter (includes reset option "All")
    html.Div([
        html.Label("Filter by Animal Type:"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'All', 'value': 'All'},  # Reset view option
                {'label': 'Dog', 'value': 'Dog'},
                {'label': 'Cat', 'value': 'Cat'}
            ],
            value='All',  # Default: show all animals
            labelStyle={'display': 'inline-block'}
        )
    ]),

    # Rescue type filter (custom categories)
    html.Div([
        html.Label("Filter by Rescue Type:"),
        dcc.RadioItems(
            id='rescue-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster Rescue', 'value': 'Disaster Rescue'},
                {'label': 'Individual Tracking', 'value': 'Individual Tracking'}
            ],
            value='Water Rescue',  # Default selection
            labelStyle={'display': 'inline-block'}
        )
    ]),

    # Dashboard header and logo
    html.Center(html.B(html.H1('CS-340 Dashboard - Jose Medina'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'height': '150px'}),
    html.Hr(),

    # Main data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,  # Show 10 records per page
        style_table={'overflowX': 'auto'},  # Enable horizontal scrolling
        style_cell={'textAlign': 'left'}   # Align cell content to the left
    ),

    html.Br(),
    html.Hr(),

    # Visualizations: Graph and Map side-by-side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),  # Outcome type chart
        html.Div(id='map-id', className='col s12 m6')     # Geolocation map
    ])
])


In [4]:
# Database connection using your AnimalShelter class
from animal_crud import AnimalShelter

username = "aacuser"
password = "123456"

# Connect to MongoDB using AnimalShelter CRUD module
db = AnimalShelter(username, password)
df = pd.DataFrame.from_records(db.read({}))

# Drop MongoDB _id field if it exists
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)


In [5]:
from dash.dependencies import Input, Output

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value'),
     Input('rescue-type', 'value')]
)
def update_dashboard(filter_type, rescue_type):
    query = {}

    # Filter by animal type unless "All"
    if filter_type != 'All':
        query["animal_type"] = filter_type

    # Apply rescue type filter ONLY if animal type is Dog
    if filter_type == "Dog":
        if rescue_type == "Water Rescue":
            query["breed"] = {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}
        elif rescue_type == "Mountain or Wilderness Rescue":
            query["breed"] = {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                                      "Siberian Husky", "Rottweiler"]}
        elif rescue_type == "Disaster Rescue":
            query["breed"] = {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                                      "Bloodhound", "Rottweiler"]}
        elif rescue_type == "Individual Tracking":
            query["breed"] = {"$in": ["Labrador Retriever Mix", "German Shepherd", "Golden Retriever",
                                      "Bloodhound", "Labrador Retriever"]}

    # Pull filtered records from database
    filtered_df = pd.DataFrame.from_records(db.read(query))

    # Drop _id if it exists
    if '_id' in filtered_df.columns:
        filtered_df.drop(columns=['_id'], inplace=True)

    return filtered_df.to_dict('records')


In [6]:
# Callback to update the outcome type pie chart based on the current filtered table data
@app.callback(
    Output('graph-id', "children"),                      # Output to the graph section of the dashboard
    [Input('datatable-id', "derived_virtual_data")]      # Triggered when the data in the table changes
)
def update_graphs(view_data):
    # If no data is available, return a placeholder message
    if view_data is None or len(view_data) == 0:
        return html.P("No data available to display chart.")

    # Convert table data (list of dicts) into a pandas DataFrame
    dff = pd.DataFrame.from_dict(view_data)

    # Return a Plotly pie chart of the 'outcome_type' column
    return [
        dcc.Graph(
            figure=px.pie(
                dff,
                names='outcome_type',
                title='Outcome Type Distribution'
            )
        )
    ]

In [7]:
# Callback to apply conditional styling to the selected column(s) in the data table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),    # Updates the style for selected columns
    [Input('datatable-id', 'selected_columns')]          # Triggered when a column is selected
)
def update_styles(selected_columns):
    # If no columns are selected, return an empty style
    if not selected_columns:
        return []

    # Highlight each selected column with a light blue background
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


In [8]:
# Callback to update the map component based on selected row(s) from the data table
@app.callback(
    Output('map-id', 'children'),  # Output to the map container
    [Input('datatable-id', 'derived_virtual_data'),         # Table data as currently filtered
     Input('datatable-id', 'derived_virtual_selected_rows')]  # Index of the selected row(s)
)
def update_map(viewData, index):
    # If no data is available, return a message
    if not viewData:
        return html.P("No data to display on the map.")

    # Convert the table data into a DataFrame
    dff = pd.DataFrame.from_dict(viewData)

    # Check for empty data or missing columns required for mapping
    if dff.empty or dff.shape[1] < 15:
        return html.P("Data missing required map columns.")

    # Use the first selected row index; default to the first row if none selected
    row = index[0] if index else 0

    # Attempt to safely extract the map-related fields (latitude, longitude, breed, name)
    try:
        lat = dff.iloc[row, 13]    # Latitude
        lon = dff.iloc[row, 14]    # Longitude
        breed = dff.iloc[row, 4]   # Breed (used for tooltip)
        name = dff.iloc[row, 9]    # Name (used for popup)
    except IndexError:
        return html.P("Error accessing map data. Please ensure valid selection.")

    # Render a Dash Leaflet map centered on Austin, TX with a marker for the selected animal
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),  # Base map layer
            dl.Marker(position=[lat, lon], children=[  # Marker positioned by animal's coordinates
                dl.Tooltip(breed),                     # Show breed on hover
                dl.Popup([                             # Show detailed info on click
                    html.H1("Animal Name"),
                    html.P(name)
                ])
            ])
        ])
    ]


In [9]:
app.run_server(debug=True)


Dash app running on http://127.0.0.1:17513/
